In [1]:
import os
from glob import glob


import tensorflow as tf
import tensorflow_hub as hub

from google.colab import drive
import os
import torch

import numpy as np
import matplotlib.pyplot as plt
import time
import math

drive.mount('/content/drive')
root_path = '/content/drive/My Drive/Colab Notebooks/Sentiment Classifier/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers

In [3]:
dir_path = 'XFORMAL'
file_path = dir_path + '.zip'
if not os.path.isdir(dir_path):
    if not os.path.isfile(file_path):
        !wget -O XFORMAL.zip "https://docs.google.com/uc?export=download&id=1cF8AXSQ1OZhgIbaIBWc3n8xgj2_prY8C"
    !7z x XFORMAL.zip

    for i in glob('XFORMAL/gyafc_translated/*/*/*/'):
        list_files = os.listdir(i)
        informal_path = os.path.join(i, '1_informal')
        os.mkdir(informal_path)
        formal_path = os.path.join(i, '1_formal')
        os.mkdir(formal_path)
        for j in list_files:
            if 'informal' in j:
                os.rename(
                    os.path.join(i, j),
                    os.path.join(informal_path, j),
                )
            else:
                os.rename(
                    os.path.join(i, j),
                    os.path.join(formal_path, j),
                )
        os.rename(informal_path, os.path.join(i, 'informal'))
        os.rename(formal_path, os.path.join(i, 'formal'))

In [4]:
def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    # Note: You'll use indexing here instead of tuple unpacking to enable this 
    # to work in a TensorFlow graph.
    return parts[-2]

In [5]:
class_names = ['formal', 'informal']

In [6]:
def labeler(example, example_path):
    return example, tf.argmax(get_label(example_path) == class_names)

def labeler(example_path):
    return get_label(example_path)

In [7]:
data_path = 'XFORMAL/gyafc_translated/*/*/{}/*/*'

In [8]:
BATCH_SIZE = 512
BUFFER_SIZE = 200000

In [9]:
from transformers import BertTokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

def get_files(path_dataset,):


    data = {"fr" : {"formal" : [], "informal" : []},
            "pt" : {"formal" : [], "informal" : []},
            "en" : {"formal" : [], "informal" : []},
            "it" : {"formal" : [], "informal" : []},
            "ru" : {"formal" : [], "informal" : []}
            }

    for file_name in glob(path_dataset):
      with open(file_name, "r") as f:
        content = f.readlines()

        # print(f"language: {file_name[25:27]}, type: {file_name}, lenght: {len(content)},  label: {get_label(file_name)}")
        

        data[file_name[25:27]][str(get_label(file_name).numpy())[2:-1]] += content

    # data = {
    #         "fr" : {"formal": list(set(data["fr"]["formal"])), "informal": list(set(data["fr"]["informal"]))},
    #         "pt" : {"formal": list(set(data["pt"]["formal"])), "informal": list(set(data["pt"]["informal"]))},
    #         "en" : {"formal": list(set(data["en"]["formal"])), "informal": list(set(data["en"]["informal"]))},
    #         "it" : {"formal": list(set(data["it"]["formal"])), "informal": list(set(data["it"]["informal"]))},
    #         "ru" : {"formal": list(set(data["ru"]["formal"])), "informal": list(set(data["ru"]["informal"]))}
    #         }

    # data = {
    #         "fr" : {"formal": [sentence for sentence in list(set(data["fr"]["formal"])) if list(bert_tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")["input_ids"].shape)[1]<=50],
    #                 "informal": [sentence for sentence in list(set(data["fr"]["informal"])) if list(bert_tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")["input_ids"].shape)[1]<=50]},
            
    #         "pt" : {"formal": [sentence for sentence in list(set(data["pt"]["formal"])) if list(bert_tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")["input_ids"].shape)[1]<=50],
    #                 "informal": [sentence for sentence in list(set(data["pt"]["informal"])) if list(bert_tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")["input_ids"].shape)[1]<=50]},
            
    #         "en" : {"formal": [sentence for sentence in list(set(data["en"]["formal"])) if list(bert_tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")["input_ids"].shape)[1]<=50],
    #                 "informal": [sentence for sentence in list(set(data["en"]["informal"])) if list(bert_tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")["input_ids"].shape)[1]<=50]},
            
    #         "it" : {"formal": [sentence for sentence in list(set(data["it"]["formal"])) if list(bert_tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")["input_ids"].shape)[1]<=50],
    #                 "informal": [sentence for sentence in list(set(data["it"]["informal"])) if list(bert_tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")["input_ids"].shape)[1]<=50]},
            
    #         "ru" : {"formal": [sentence for sentence in list(set(data["ru"]["formal"])) if list(bert_tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")["input_ids"].shape)[1]<=50],
    #                 "informal": [sentence for sentence in list(set(data["ru"]["informal"])) if list(bert_tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")["input_ids"].shape)[1]<=50]},
    #         }

    data = {
            "fr" : {"formal": [sentence for sentence in list(set(data["fr"]["formal"])) if len(sentence) <=150],
                    "informal": [sentence for sentence in list(set(data["fr"]["informal"])) if len(sentence) <=150]},
            
            "pt" : {"formal": [sentence for sentence in list(set(data["pt"]["formal"])) if len(sentence) <=150],
                    "informal": [sentence for sentence in list(set(data["pt"]["informal"])) if len(sentence) <=150]},
            
            "en" : {"formal": [sentence for sentence in list(set(data["en"]["formal"])) if len(sentence) <=150],
                    "informal": [sentence for sentence in list(set(data["en"]["informal"])) if len(sentence) <=150]},
            
            "it" : {"formal": [sentence for sentence in list(set(data["it"]["formal"])) if len(sentence) <=150],
                    "informal": [sentence for sentence in list(set(data["it"]["informal"])) if len(sentence) <=150]}, 
            # "ru" : {"formal": [sentence for sentence in list(set(data["ru"]["formal"])) if len(sentence) <=150],
            #         "informal": [sentence for sentence in list(set(data["ru"]["informal"])) if len(sentence) <=150]}, 
            }

            

    return data


In [10]:


train_data  = get_files(data_path.format('train'))

for language in train_data:
  print(f"language: {language}, # formal: {len(train_data[language]['formal'])}, # informal: {len(train_data[language]['informal'])}")
print('---------------------------')

validation_data = get_files(data_path.format('test'))
for language in validation_data:
  print(f"language: {language}, # formal: {len(validation_data[language]['formal'])}, # informal: {len(validation_data[language]['informal'])}")
print('---------------------------')

test_data = get_files(data_path.format('tune'))
for language in train_data:
  print(f"language: {language}, # formal: {len(test_data[language]['formal'])}, # informal: {len(test_data[language]['informal'])}")

all_data = [train_data, validation_data, test_data]


language: fr, # formal: 103209, # informal: 100173
language: pt, # formal: 103630, # informal: 100260
language: en, # formal: 103979, # informal: 100386
language: it, # formal: 103536, # informal: 100236
---------------------------
language: fr, # formal: 10326, # informal: 9016
language: pt, # formal: 10418, # informal: 9016
language: en, # formal: 10710, # informal: 9031
language: it, # formal: 10402, # informal: 9001
---------------------------
language: fr, # formal: 21431, # informal: 19354
language: pt, # formal: 21602, # informal: 19363
language: en, # formal: 22132, # informal: 19448
language: it, # formal: 21483, # informal: 19349


In [11]:
from random import shuffle
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

class SentenceDataset(Dataset):
    def __init__(self, datasets):
      self.formal = []
      self.informal = []
      if isinstance(datasets, list):
        for dataset in datasets:
            self.formal += [sentence for sentence in dataset['formal']]
            self.informal += [sentence for sentence in dataset['informal']]  
      else:
        for language in datasets:
          self.formal += [sentence for sentence in datasets[language]['formal']]
          self.informal += [sentence for sentence in datasets[language]['informal']]        

      print(f"formal sentences: {len(self.formal)}")
      print(f"informal sentences: {len(self.informal)}")

      self.tuples = []
      self.tuples += [(sentence,0) for sentence in self.formal] 
      self.tuples += [(sentence,1) for sentence in self.informal]
      shuffle(self.tuples)

    def __len__(self):
        return len(self.tuples)

    def __getitem__(self, idx):
        return self.tuples[idx]

cuda


In [12]:
from torch import nn
class char_BiLSTM(nn.Module):
    def __init__(self, embedding_dim=32, hidden_dim=128, lstm_layer=1, output=2, train = True):
        
        super(char_BiLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        
        # load pre-trained embeddings
        self.embedding = nn.Embedding(len(char_vocab)+1, embedding_dim)
        # embeddings are not fine-tuned
        self.embedding.weight.requires_grad = train
        
        # RNN layer with LSTM cells
        # OR self.lstm = NaiveLSTM(input_sz = self.embedding.embedding_dim, hidden_sz = hidden_dim)
        self.lstm = nn.LSTM(input_size=self.embedding.embedding_dim,
                            hidden_size=hidden_dim,
                            num_layers=lstm_layer, 
                            bidirectional=True,
                            dropout = 0.5)
        # dense layer
        self.output = nn.Linear(hidden_dim*2, output)
    
    def forward(self, sents):
        x = self.embedding(sents[0])
        
        # the original dimensions of torch LSTM's output are: (seq_len, batch, num_directions * hidden_size)
        lstm_out, _ = self.lstm(x)
        
        # reshape to get the tensor of dimensions (seq_len, batch, num_directions, hidden_size)
        lstm_out = lstm_out.view(x.shape[0], -1, 2, self.hidden_dim)#.squeeze(1)
        

        # we take the last hidden state of the forward LSTM and the first hidden state of the backward LSTM
        dense_input = torch.cat((lstm_out[-1, :, 0, :], lstm_out[0, :, 1, :]), dim=1)
        
        y = self.output(dense_input)#.view([1, 2])
        return [y]

text = []
for set_data in all_data:
  for language in set_data:
    text += set_data[language]['formal'] 
    text += set_data[language]['informal'] 
text = [sentence for sentence in text if len(sentence) <= 150]

print(f"total sentences: {len(text)}")

MAX_LEN = max([len(sentence) for sentence in text])
PRUNE_TOKENS_LESS_THAN = 0

print("maxlen :", str(MAX_LEN))


tokens = set.union(*[set(sentence) for sentence in text])
token_stats = {token :  0 for token in tokens}
text = ' '.join(text)
for token in list(tokens):
  token_stats[token] = text.count(token)


tokens = [key for (key,value) in token_stats.items() if value >= PRUNE_TOKENS_LESS_THAN]
tokens = sorted(tokens)
tokens = {key:id for (id,key) in enumerate(tokens)}
tokens["<EMP>"] = len(tokens)
tokens["<UNK>"] = len(tokens)
print(f"total tokens: {len(list(tokens))}")

def char_pipeline(text, length):
  chars = [char for char in text]
  text = char_vocab(chars)
  text += [char_vocab["<EMP>"]] * (length - len(text))
  return torch.tensor(text, dtype=torch.int64)


def char_collate_batch(batch):
  max_length = len(max(batch, key=lambda t: len(t[0]))[0])
  label_list, text_list = [], []
  for (_text, _label) in batch:
        label_list.append(_label)
        processed_text = char_pipeline(_text, max_length)
        text_list.append(processed_text)
  label_list = torch.tensor(label_list, dtype=torch.int64)
  text_list = torch.stack(text_list, dim=-1)
  # print("Text List ",list(text_list.size()))
  # print("Label List ",list(label_list.size()))
  return label_list.to(device), text_list.to(device)

total sentences: 1057491
maxlen : 150
total tokens: 218


In [13]:

def char_pipeline(text, length):
  chars = [char for char in text]
  text = char_vocab(chars)
  text += [char_vocab["<EMP>"]] * (length - len(text))
  return torch.tensor(text, dtype=torch.int64)


def char_collate_batch(batch):
  max_length = len(max(batch, key=lambda t: len(t[0]))[0])
  label_list, text_list = [], []
  for (_text, _label) in batch:
        label_list.append(_label)
        processed_text = char_pipeline(_text, max_length)
        text_list.append(processed_text)
  label_list = torch.tensor(label_list, dtype=torch.int64)
  text_list = torch.stack(text_list, dim=-1)
  # print("Text List ",list(text_list.size()))
  # print("Label List ",list(label_list.size()))
  return [text_list], [label_list]

In [14]:
from torchtext.vocab import vocab

char_vocab = vocab(tokens)
char_vocab.set_default_index(len(tokens)-1)

In [15]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove.6B.zip

In [16]:
#@title Word BiLSTM model

# from torchtext.data.utils import get_tokenizer
# from torchtext.vocab import build_vocab_from_iterator
# from nltk import wordpunct_tokenize


# class word_BiLSTM(nn.Module):
#     def __init__(self, embeddings, trainable = False, hidden_dim=128, lstm_layer=1, output=2):
        
#         super(word_BiLSTM, self).__init__()
#         self.hidden_dim = hidden_dim
        
#         # load pre-trained embeddings
#         self.embedding = nn.Embedding.from_pretrained(embeddings)
#         # embeddings are not fine-tuned
#         self.embedding.weight.requires_grad = trainable
        
#         # RNN layer with LSTM cells
#         # OR self.lstm = NaiveLSTM(input_sz = self.embedding.embedding_dim, hidden_sz = hidden_dim)
#         self.lstm = nn.LSTM(input_size=self.embedding.embedding_dim,
#                             hidden_size=hidden_dim,
#                             num_layers=lstm_layer, 
#                             bidirectional=True)
#         # dense layer
#         self.output = nn.Linear(hidden_dim*2, output)
    
#     def forward(self, sents):
#         x = self.embedding(sents)
        
#         # the original dimensions of torch LSTM's output are: (seq_len, batch, num_directions * hidden_size)
#         lstm_out, _ = self.lstm(x)
        
#         # reshape to get the tensor of dimensions (seq_len, batch, num_directions, hidden_size)
#         lstm_out = lstm_out.view(x.shape[0], -1, 2, self.hidden_dim)#.squeeze(1)
        
#         # lstm_out[:, :, 0, :] -- output of the forward LSTM
#         # lstm_out[:, :, 1, :] -- output of the backward LSTM
#         # we take the last hidden state of the forward LSTM and the first hidden state of the backward LSTM
#         dense_input = torch.cat((lstm_out[-1, :, 0, :], lstm_out[0, :, 1, :]), dim=1)
        
#         y = self.output(dense_input)#.view([1, 2])
#         return y



# text = formal + informal
# word_vocab = set()
# for sentence in text:
#   words = wordpunct_tokenize(sentence)
#   word_vocab.update(words)

# def load_embeddings(emb_path, vocab):
#     clf_embeddings = {}
#     emb_vocab = set()
#     for line in open(emb_path):
#         line = line.strip('\n').split()
#         word, emb = line[0], line[1:]
#         emb = [float(e) for e in emb]
#         if word in vocab:
#             clf_embeddings[word] = emb
#     for w in vocab:
#         if w in clf_embeddings:
#             emb_vocab.add(w)
#     word2idx = {w: idx for (idx, w) in enumerate(emb_vocab)}
#     max_val = max(word2idx.values())
    
#     word2idx['UNK'] = max_val + 1
#     word2idx['EOS'] = max_val + 2
#     emb_dim = len(list(clf_embeddings.values())[0])
#     clf_embeddings['UNK'] = [0.0 for i in range(emb_dim)]
#     clf_embeddings['EOS'] = [0.0 for i in range(emb_dim)]
    
#     embeddings = [[] for i in range(len(word2idx))]
#     for w in word2idx:
#         embeddings[word2idx[w]] = clf_embeddings[w]
#     embeddings = torch.Tensor(embeddings)
#     return embeddings, word2idx

# word_embeddings, word_vocab = load_embeddings('glove.6B.300d.txt', word_vocab)    

# def text_pipeline_words(text, length):
#   chars = wordpunct_tokenize(text)  
#   text = [word_vocab.get(char, word_vocab['UNK']) for char in chars]
#   text += [word_vocab["EOS"]] * (length - len(text))
#   # print(torch.tensor(text, dtype=torch.int64))
#   return torch.tensor(text, dtype=torch.int64)

# def collate_batch_words(batch):
#   sentences = [text for (text, label) in batch]
#   max_length = 0

#   for sentence in sentences:
#     if len(wordpunct_tokenize(sentence)) > max_length:
#       max_length = len(wordpunct_tokenize(sentence))

#   label_list, text_list = [], []
#   for (_text, _label) in batch:
#         label_list.append(_label)
#         processed_text = text_pipeline_words(_text, max_length)
#         text_list.append(processed_text)
#   label_list = torch.tensor(label_list, dtype=torch.int64)
#   text_list = torch.stack(text_list, dim=-1)
#   return [text_list], [label_list]



In [17]:
#@title BERT model

from transformers import DistilBertModel, DistilBertTokenizer
from transformers import BertTokenizer, BertModel
from transformers import RobertaModel, RobertaTokenizer

class BERT_Model(nn.Module):
    def __init__(self,bert_model, layer2, layer3, Train = False):
        super(BERT_Model, self).__init__()

        self.bert = bert_model
        for param in self.bert.parameters():
            param.requires_grad = Train

        

        self.out = nn.Sequential(
          nn.Linear(self.bert.config.hidden_size, layer2),
          nn.Linear(layer2, layer3),
          nn.Linear(layer3, 2)
        )

    def forward(self, input):
        id = input[0]
        mask = input[1]

        bert_output = self.bert(id, attention_mask = mask)
        output = self.out(bert_output[1])
        return [output]

class Distil_BERT_Model(nn.Module):
    def __init__(self, layer2, layer3, Train = False):
        super(Distil_BERT_Model, self).__init__()

        self.bert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        for param in self.bert.parameters():
            param.requires_grad = Train
        
        self.out = nn.Sequential(
          nn.Linear(self.bert.config.hidden_size, layer2),
          nn.Linear(layer2, layer3),
          nn.Linear(layer3, 2)
        )
    def forward(self, input):
        id = input[0]
        mask = input[1]

        bert_output = self.bert(id, attention_mask = mask)

        #take the last value from the bert output
        output = self.out(torch.flatten(bert_output[0].narrow(1,-1,1),1))
        return [output]

class RoBERTa_Model(nn.Module):
    def __init__(self, layer2, layer3, Train = False):
        super(RoBERTa_Model, self).__init__()

        self.bert = RobertaModel.from_pretrained('roberta-base')
        for param in self.bert.parameters():
            param.requires_grad = Train
        
        self.out = nn.Sequential(
          nn.Linear(self.bert.config.hidden_size, layer2),
          nn.Linear(layer2, layer3),
          nn.Linear(layer3, 2)
        )

    def forward(self, input):
        id = input[0]
        mask = input[1]

        bert_output = self.bert(id, attention_mask = mask)

        #take the last value from the bert output
        output = self.out(torch.flatten(bert_output[0].narrow(1,-1,1),1))
        return [output]

bert_cased_tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
bert_uncased_tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")

Disitl_bert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
RoBERTa_tokeniser = RobertaTokenizer.from_pretrained("roberta-base")

def Distil_BERT_collate_batch(batch):
  sentences = [text for (text, label) in batch]
  labels = [label for (text, label) in batch]

  # for sentence in sentences:
  temp = Disitl_bert_tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
  text_list = temp["input_ids"]
  mask_list = temp["attention_mask"]
  label_list = torch.tensor(labels, dtype=torch.int64)

  # text_list = torch.stack(text_list, dim=0)
  return  [text_list, mask_list], [label_list]

def BERT_cased_collate_batch(batch):
  sentences = [text for (text, label) in batch]
  labels = [label for (text, label) in batch]

  # for sentence in sentences:
  temp = bert_cased_tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
  text_list = temp["input_ids"]
  mask_list = temp["attention_mask"]
  label_list = torch.tensor(labels, dtype=torch.int64)

  # text_list = torch.stack(text_list, dim=0)
  return  [text_list, mask_list], [label_list]

def BERT_uncased_collate_batch(batch):
  sentences = [text for (text, label) in batch]
  labels = [label for (text, label) in batch]

  # for sentence in sentences:
  temp = bert_uncased_tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
  text_list = temp["input_ids"]
  mask_list = temp["attention_mask"]
  label_list = torch.tensor(labels, dtype=torch.int64)

  # text_list = torch.stack(text_list, dim=0)
  return  [text_list, mask_list], [label_list]

def RoBERTa_collate_batch(batch):
  sentences = [text for (text, label) in batch]
  labels = [label for (text, label) in batch]

  # for sentence in sentences:
  temp = RoBERTa_tokeniser(sentences, padding=True, truncation=True, return_tensors="pt")
  text_list = temp["input_ids"]
  mask_list = temp["attention_mask"]
  label_list = torch.tensor(labels, dtype=torch.int64)

  # text_list = torch.stack(text_list, dim=0)
  return  [text_list, mask_list], [label_list]


In [18]:
#@title Train Methods

import gc
def cleanup():
    gc.collect()
    torch.cuda.empty_cache()


def train(dataloader, log_interval):
    model.train()
    final_loss, accuracy, count = 0, 0, 0
    interval_time = time.time()
    epoch_time = interval_time
    for idx, (inputs, outputs) in enumerate(dataloader):
      cleanup()
      optimizer.zero_grad()

      inputs = [input.to(device) for input in inputs]
      outputs = [output.to(device) for output in outputs]
      predicted_outputs = model(inputs)
      for output, predicted_output in zip(outputs, predicted_outputs):
        loss = criterion(predicted_output, output)
        loss.backward()
        final_loss += loss.item()
      torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
      optimizer.step()

      accuracy += (predicted_outputs[0].argmax(1) == outputs[0]).sum().item()
      count += outputs[0].size(0)

      if idx % log_interval == 0 and idx > 0:
        elapsed = time.time() - interval_time
        print(f'| epoch {epoch} \| {idx}/{len(dataloader)} batches \| accuracy {accuracy/count} \| time elapsed {elapsed} \| batches/second {log_interval/elapsed}')
        interval_time = time.time()
      


    cleanup()
    return  final_loss/count, accuracy/count, time.time() -epoch_time



def evaluate(dataloader):
  model.eval()
  accuracy, loss, count = 0,0,0
  with torch.no_grad():
    for idx, (inputs, outputs) in enumerate(dataloader):
      cleanup()

      inputs = [input.to(device) for input in inputs]
      outputs = [output.to(device) for output in outputs]

      predicted_outputs = model(inputs)

      for output, predicted_output in zip(outputs, predicted_outputs):
        loss += criterion(predicted_output, output)

      accuracy += (predicted_outputs[0].argmax(1) == outputs[0]).sum().item()
      count += outputs[0].size(0)

      cleanup()
  return loss/count, accuracy/count


def get_labels(dataloader):
    model.eval()
    err_num = 0
    labels = []
    predicted_labels =[]
    with torch.no_grad():
      for idx, (inputs, outputs) in enumerate(dataloader): 

        cleanup()

        inputs = [input.to(device) for input in inputs]
        outputs = [output.to(device) for output in outputs]

        predicted_label = model(inputs)
        predicted_label = torch.argmax(predicted_label[0], 1)
        predicted_label = predicted_label.cpu().detach().numpy().tolist()
        label = outputs[0].cpu().detach().numpy()

        predicted_labels.extend(predicted_label)
        labels.extend(label)

        cleanup()
    return labels, predicted_labels

In [19]:
#@title Logs

from sklearn.metrics import confusion_matrix, classification_report
!pip install openpyxl

import openpyxl as opx
from math import floor

def number_to_column(num):
  if num < 26:
    return chr(num+65)
  else:
    return chr(floor(num/26)+64) + chr((num%26)+65)

class logs():
  def __init__(self, folder, model_name, columns):
    wb = opx.load_workbook(filename = folder + "/logs_multilingual.xlsx")
    sheet_columns = {}
    if model_name in wb.sheetnames:
      print(f"Found a sheet for the model {model_name}")
      sheet = wb[model_name]
      idx = 1
      while sheet.cell(row = 1, column = idx).value:
        print(sheet.cell(row = 1, column = idx).value)
        sheet_columns[sheet.cell(row = 1, column = idx).value] = idx
        idx += 1
      print(sheet_columns)
      self.check_columns(columns, sheet_columns)
    else:
      self.check_columns(columns, sheet_columns)      
      print(f"worksheet for {model_name} not found, creating new worksheet")
      sheet = wb.create_sheet(model_name)
      for idx, column in enumerate(columns):
        sheet[f"{number_to_column(idx)}1"] = column
      wb.save(filename = folder + "/logs_multilingual.xlsx")
    if sheet_columns == {}:
      for idx, column in enumerate(columns):
        sheet_columns["colums"] = idx + 1
    
    self.model_name = model_name
    self.sheet_columns = sheet_columns
    self.folder = folder
    
    self.train_acc = []
    self.val_acc = []
    self.train_loss = []
    self.val_loss = []

    self.test_acc = 0

  def check_columns(self, columns, sheet_columns):
    required_columns = ["Epochs", "Batch size",  "Description", "epoch time"]
    if sheet_columns != {}:
      sheet_columns = list(sheet_columns.keys())
      if sheet_columns  != columns:
        print(sheet_columns)
        print("---------")
        print(columns)
        raise NameError("The provided columns do not align with columns in the sheet")
    if not set(required_columns).issubset(columns):
      raise NameError("The provided columns lack some of the required columns")

  def reserve_id(self):
    wb = opx.load_workbook(filename = self.folder + "/logs_multilingual.xlsx")
    sheet = wb[self.model_name]
    idx = 1
    column = self.sheet_columns["Description"]
    while sheet.cell(row = idx, column = column).value:
      idx += 1
    sheet[f"{chr(column+64)}{idx}"] = "reserved"
    wb.save(filename = self.folder + "/logs_multilingual.xlsx")    

    self.model_folder = self.folder + f"/data_logs/{self.model_name}#{idx}"

    return idx

  def write_data(self, data, idx):
    wb = opx.load_workbook(filename = self.folder + "/logs_multilingual.xlsx")
    sheet = wb[self.model_name]
    for (row_name, data_point) in data:
      sheet[f"{number_to_column(self.sheet_columns[row_name]-1)}{idx}"] = data_point
    wb.save(filename = self.folder + "/logs_multilingual.xlsx")    


    # self.logdir = self.folder + f"/data_logs/{self.model_name}#{self.}"
    tensorboard_writer = SummaryWriter(self.model_folder)       

    for epoch, val in enumerate(self.train_acc):
      tensorboard_writer.add_scalar('Accuracy/train', val, epoch)    

    for epoch, val in enumerate(self.val_acc):
      tensorboard_writer.add_scalar('Accuracy/validation', val, epoch)    

    for epoch, val in enumerate(self.train_loss):
      tensorboard_writer.add_scalar('Loss/train', val, epoch)    

    for epoch, val in enumerate(self.val_loss):
      tensorboard_writer.add_scalar('Loss/validation', val, epoch)

    tensorboard_writer.flush()
    tensorboard_writer.close()          

In [20]:
test = [train_data[temp] for temp in train_data.keys()]
print([train_data[temp].keys() for temp in train_data.keys()])


[dict_keys(['formal', 'informal']), dict_keys(['formal', 'informal']), dict_keys(['formal', 'informal']), dict_keys(['formal', 'informal'])]


In [ ]:
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import csv
cleanup()


# Hyperparameters
data = {}
EPOCHS = 10  # epoch
LR = 5  # learning rate
BATCH_SIZE = 512 # batch size for training
MAX_TRAIN_STEPS = 1
LOG_INTERVAL = 200

NAME = "char_BiLSTM"
DESCRIPTION = "multilingual char BiLSTM"
EMBEDDING = 128
HIDDEN_STATE = 128
LAYERS = 2

DROPOUT = 0.5
languages = ['fr','pt','it','all']


architectures = {}
# architectures["char_BiLSTM"] = {
#     "Folder" : "folder",
#     "Description" : "Char BiLSTM model",
#     "model" : char_BiLSTM(embedding_dim=128, hidden_dim=128, lstm_layer=2, output=2, train = True).to(device),
#     "collate_batch" : char_collate_batch
# }    
architectures["BERT1"] = {
    "Folder" : "folder",
    "Description" : "Cased Multilingual BERT model",
    # "model" : BERT_Model(BertModel.from_pretrained('bert-base-multilingual-cased'), 1024, 1024, Train = False).to(device),
    "collate_batch" : BERT_cased_collate_batch
}    
# architectures["BERT2"] = {
#     "Folder" : "folder",   
#     "Description" : "Uncased Multilingual BERT model",
#     "model" : BERT_Model(BertModel.from_pretrained('bert-base-multilingual-uncased'), 1024, 1024, Train = False).to(device),
#     "collate_batch" : BERT_uncased_collate_batch
# }   
  


for architecture in architectures.keys():
  #--------------------
  #Create Logs
  #--------------------
  data["Name"] = "temp"
  data["Epochs"] = EPOCHS
  data["Batch size"] = BATCH_SIZE
  # data["Embeddings Trainable"] = "True"
  # data["Layers"] = LAYERS
  # data["hidden state"] = HIDDEN_STATE


  data["epoch time"] = "temp"
  data["Description"] = architectures[architecture]["Description"]

  for language in languages:
    data[f"{language}-Loss"] = "temp"
    
    data[f"{language}-train accuracy"] = "temp"
    data[f"{language}-validation accuracy"] = "temp"
    data[f"{language}-test accuracy"] = "temp"

    data[f'{language}-formal-precision'] = "temp"
    data[f'{language}-formal-recall'] = "temp"
    data[f'{language}-formal-f1-score'] = "temp"

    data[f'{language}-informal-precision'] = "temp"
    data[f'{language}-informal-recall'] = "temp"
    data[f'{language}-informal-f1-score'] = "temp"


  log = logs("/content/drive/My Drive/Colab Notebooks/Sentiment Classifier", architectures[architecture]['Folder'], list(data.keys()))
  id = log.reserve_id()

  try: 
      os.mkdir(log.model_folder) 
  except OSError as error: 
      print(error)  
  for language in languages:
    if language == "all":
      train_dataset  = SentenceDataset([train_data[temp] for temp in train_data.keys()])
      val_dataset  = SentenceDataset([validation_data[temp] for temp in validation_data.keys()])
      test_dataset  = SentenceDataset([test_data[temp] for temp in test_data.keys()])
    else:
      train_dataset  = SentenceDataset({language:train_data[language]})
      val_dataset  = SentenceDataset({language:validation_data[language]})
      test_dataset  = SentenceDataset({language:test_data[language]})


    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=architectures[architecture]['collate_batch'], num_workers=4)
    valid_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=architectures[architecture]['collate_batch'], num_workers=4)
    test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=architectures[architecture]['collate_batch'], num_workers=4)


    model = BERT_Model(BertModel.from_pretrained('bert-base-multilingual-cased'), 1024, 1024, Train = False).to(device)


    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=LR)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.8)

    total_accu = None      

    epoch_train_accuracy = []
    epoch_val_accuracy = []
    epoch_time_list = []

    for epoch in range(0, EPOCHS):



      epoch_start_time = time.time()
      epoch_loss, epoch_acc, epoch_time = train(train_dataloader, LOG_INTERVAL)
      validation_loss, validation_acc = evaluate(valid_dataloader)



      if epoch >= MAX_TRAIN_STEPS:
        STEP_SCHEDULER = True
        for i in range(MAX_TRAIN_STEPS):
          if epoch_val_accuracy[((i+1)*(-1))] >= validation_acc:
            STEP_SCHEDULER = True
        if STEP_SCHEDULER == True:
          scheduler.step()
          # for g in optimizer.param_groups:
          #     g['lr'] =  g['lr']*0.1
          print("SCHEDULER STEP")

      if len(epoch_val_accuracy) >= 1:
        if validation_acc > max(epoch_val_accuracy):
          torch.save(model, (log.model_folder + "/model"))
          print("SAVED BEST VALIDATION MODEL WITH ACCURACY: {}".format(validation_acc))
      else:
        torch.save(model, (log.model_folder + "/model"))
        print("SAVED BEST VALIDATION MODEL WITH ACCURACY: {}".format(validation_acc))

      epoch_train_accuracy.append(epoch_acc)
      epoch_val_accuracy.append(validation_acc)



      # if total_accu is not None and total_accu > validation_acc:
      #   scheduler.step()
      # else:
      #   total_accu = validation_acc
      print('-' * 59)
      print('| end of epoch {:3d} | time: {:5.2f}s | '
            'valid accuracy {:8.3f} '.format(epoch,
                                            time.time() - epoch_start_time,
                                            validation_acc))
      print('-' * 59)


      log.train_loss.append(epoch_loss)
      log.val_loss.append(validation_loss)

      log.train_acc.append(epoch_acc)
      log.val_acc.append(validation_acc)

    model = torch.load((log.model_folder + "/model"))

    _, accu_test = evaluate(test_dataloader)


    #--------------------
    #Write Down The data for the logs
    #--------------------
    data["Name"] = f"{log.model_name}#{id}"    
    data[f"{language}-Loss"] = epoch_loss
    data[f"{language}-train accuracy"] = epoch_acc
    data[f"{language}-validation accuracy"] = validation_acc
    data[f"{language}-test accuracy"] = accu_test
    data["epoch time"] = epoch_time



    #--------------------
    #Get precision recall data
    #--------------------
    list_label, list_predited_label = get_labels(test_dataloader)
    result = classification_report(list_label, list_predited_label, digits=6, output_dict=True)

    data[f'{language}-formal-precision'] = result['0']['precision']
    data[f'{language}-formal-recall'] = result['0']['recall']
    data[f'{language}-formal-f1-score'] = result['0']['f1-score']

    data[f'{language}-informal-precision'] = result['1']['precision']
    data[f'{language}-informal-recall'] = result['1']['recall']
    data[f'{language}-informal-f1-score'] = result['1']['f1-score']

  #--------------------
  #Dump logs
  #--------------------
  data_tuples = [(column, data[column]) for column in list(data.keys())]
  log.write_data(data_tuples, id)




Found a sheet for the model folder
Name
Epochs
Batch size
epoch time
Description
fr-Loss
fr-train accuracy
fr-validation accuracy
fr-test accuracy
fr-formal-precision
fr-formal-recall
fr-formal-f1-score
fr-informal-precision
fr-informal-recall
fr-informal-f1-score
pt-Loss
pt-train accuracy
pt-validation accuracy
pt-test accuracy
pt-formal-precision
pt-formal-recall
pt-formal-f1-score
pt-informal-precision
pt-informal-recall
pt-informal-f1-score
it-Loss
it-train accuracy
it-validation accuracy
it-test accuracy
it-formal-precision
it-formal-recall
it-formal-f1-score
it-informal-precision
it-informal-recall
it-informal-f1-score
all-Loss
all-train accuracy
all-validation accuracy
all-test accuracy
all-formal-precision
all-formal-recall
all-formal-f1-score
all-informal-precision
all-informal-recall
all-informal-f1-score
{'Name': 1, 'Epochs': 2, 'Batch size': 3, 'epoch time': 4, 'Description': 5, 'fr-Loss': 6, 'fr-train accuracy': 7, 'fr-validation accuracy': 8, 'fr-test accuracy': 9, 'fr-fo

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g

| epoch 0 \| 200/398 batches \| accuracy 0.591758006840796 \| time elapsed 248.64073634147644 \| batches/second 0.8043734222429483
SAVED BEST VALIDATION MODEL WITH ACCURACY: 0.6699410609037328
-----------------------------------------------------------
| end of epoch   0 | time: 536.98s | valid accuracy    0.670 
-----------------------------------------------------------
| epoch 1 \| 200/398 batches \| accuracy 0.7012690453980099 \| time elapsed 248.55733633041382 \| batches/second 0.8046433187316375
SCHEDULER STEP
SAVED BEST VALIDATION MODEL WITH ACCURACY: 0.7466135870127184
-----------------------------------------------------------
| end of epoch   1 | time: 537.41s | valid accuracy    0.747 
-----------------------------------------------------------
| epoch 2 \| 200/398 batches \| accuracy 0.7405064521144279 \| time elapsed 245.88915467262268 \| batches/second 0.8133746291750054
SCHEDULER STEP
SAVED BEST VALIDATION MODEL WITH ACCURACY: 0.7719470582152828
-------------------------

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


| epoch 0 \| 200/399 batches \| accuracy 0.5516752176616916 \| time elapsed 231.71225953102112 \| batches/second 0.863139483447247
SAVED BEST VALIDATION MODEL WITH ACCURACY: 0.6734074302768344
-----------------------------------------------------------
| end of epoch   0 | time: 508.35s | valid accuracy    0.673 
-----------------------------------------------------------
| epoch 1 \| 200/399 batches \| accuracy 0.6677646921641791 \| time elapsed 229.9140980243683 \| batches/second 0.8698901099087984
SCHEDULER STEP
SAVED BEST VALIDATION MODEL WITH ACCURACY: 0.7392199238448081
-----------------------------------------------------------
| end of epoch   1 | time: 511.43s | valid accuracy    0.739 
-----------------------------------------------------------
| epoch 2 \| 200/399 batches \| accuracy 0.7128614738805971 \| time elapsed 235.39231395721436 \| batches/second 0.8496454138105487
SCHEDULER STEP
SAVED BEST VALIDATION MODEL WITH ACCURACY: 0.7409694350108058
--------------------------

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


| epoch 0 \| 200/398 batches \| accuracy 0.554580612562189 \| time elapsed 232.5437626838684 \| batches/second 0.8600531688819794
SAVED BEST VALIDATION MODEL WITH ACCURACY: 0.6858217801370922
-----------------------------------------------------------
| end of epoch   0 | time: 507.84s | valid accuracy    0.686 
-----------------------------------------------------------
| epoch 1 \| 200/398 batches \| accuracy 0.6700676305970149 \| time elapsed 231.96320962905884 \| batches/second 0.8622056933934807
SCHEDULER STEP
SAVED BEST VALIDATION MODEL WITH ACCURACY: 0.7585940318507447
-----------------------------------------------------------
| end of epoch   1 | time: 507.00s | valid accuracy    0.759 
-----------------------------------------------------------
| epoch 2 \| 200/398 batches \| accuracy 0.7154753575870647 \| time elapsed 233.4915976524353 \| batches/second 0.8565618720794855
SCHEDULER STEP
-----------------------------------------------------------
| end of epoch   2 | time: 50

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


| epoch 0 \| 200/1593 batches \| accuracy 0.5836637126865671 \| time elapsed 251.63027572631836 \| batches/second 0.7948169170927858
| epoch 0 \| 400/1593 batches \| accuracy 0.6248879753740648 \| time elapsed 244.79160118103027 \| batches/second 0.8170214951618964
| epoch 0 \| 600/1593 batches \| accuracy 0.6502021370632279 \| time elapsed 249.01474952697754 \| batches/second 0.8031652758718719
| epoch 0 \| 800/1593 batches \| accuracy 0.6649281171972534 \| time elapsed 248.38311958312988 \| batches/second 0.8052076982351579
| epoch 0 \| 1000/1593 batches \| accuracy 0.6758905157342657 \| time elapsed 243.8757266998291 \| batches/second 0.8200898166719441
| epoch 0 \| 1200/1593 batches \| accuracy 0.6839775447543713 \| time elapsed 244.40761351585388 \| batches/second 0.8183051138340528
| epoch 0 \| 1400/1593 batches \| accuracy 0.6902728519807281 \| time elapsed 245.07340025901794 \| batches/second 0.8160820382327095
SAVED BEST VALIDATION MODEL WITH ACCURACY: 0.7417094455852156
-----

In [ ]:
from os import listdir
from os.path import isfile, join
drive.mount('/content/drive/')
root_path = '/content/drive/My Drive/Colab Notebooks/Sentiment Classifier/'
onlyfiles = [f for f in listdir(root_path) if isfile(join(root_path, f))]
print(filenames)